In [22]:

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from joblib import load

from sqlalchemy import create_engine
import pymssql
from datetime import datetime

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKVRM5QN9XET080C87A0"
SECRET_KEY = "L2HpFrEHZRRGvSgbyL2qRTL6yscT0vRLldLrUbir"
numstocks = 200

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model_2023-01-20.joblib")

## Rank stocks

#### Get features

In [23]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [24]:
df = pd.read_sql(
    """
    select ticker, date, bm, mom12m, roeq, mve
    from today
    where date>='2000-01-01'
    order by date, ticker
    """, 
    conn
)
conn.close()


df = df.dropna()
df = df.set_index(["date", "ticker"])
df = df.sort_values(by="mve")
df = df.iloc[:-500]

features = ["bm", "mom12m", "roeq"]

#### Predict and rank

In [25]:
df["predict"] = pipe.predict(pd.DataFrame(qt.fit_transform(df[features]), columns=features))
df["rnk"] = df.predict.rank(method="first")

## Get data from Alpaca

#### Get account equity

In [26]:
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)
account = trading_client.get_account()
equity = float(account.equity)

#### Get tradeable and shortable stocks

In [27]:
assets = trading_client.get_all_assets()

assets = [
    x for x in assets 
    if (x.asset_class[:]=='us_equity') 
    and (x.symbol in df.index) 
    and (x.status[:]=='active')
]
symbols = [x.symbol for x in assets]
tradable = [x.tradable for x in assets]
shortable = [x.shortable for x in assets]

df["tradable"]= pd.Series(tradable, index=symbols)
df["shortable"] = pd.Series(shortable, index=symbols)

C:\Users\kerry\AppData\Local\Temp\ipykernel_8072\1411646621.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df["tradable"]= pd.Series(tradable, index=symbols)
C:\Users\kerry\AppData\Local\Temp\ipykernel_8072\1411646621.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df["shortable"] = pd.Series(shortable, index=symbols)


#### Get quotes

In [28]:

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(symbol_or_symbols=df.index.to_list())
quotes = data_client.get_stock_latest_quote(params)

df["ask"] = [quotes[x].ask_price for x in df.index]
df["bid"] = [quotes[x].bid_price for x in df.index]

ValidationError: 1893 validation errors for StockLatestQuoteRequest
symbol_or_symbols
  str type expected (type=type_error.str)
symbol_or_symbols -> 0
  str type expected (type=type_error.str)
symbol_or_symbols -> 1
  str type expected (type=type_error.str)
symbol_or_symbols -> 2
  str type expected (type=type_error.str)
symbol_or_symbols -> 3
  str type expected (type=type_error.str)
symbol_or_symbols -> 4
  str type expected (type=type_error.str)
symbol_or_symbols -> 5
  str type expected (type=type_error.str)
symbol_or_symbols -> 6
  str type expected (type=type_error.str)
symbol_or_symbols -> 7
  str type expected (type=type_error.str)
symbol_or_symbols -> 8
  str type expected (type=type_error.str)
symbol_or_symbols -> 9
  str type expected (type=type_error.str)
symbol_or_symbols -> 10
  str type expected (type=type_error.str)
symbol_or_symbols -> 11
  str type expected (type=type_error.str)
symbol_or_symbols -> 12
  str type expected (type=type_error.str)
symbol_or_symbols -> 13


#### Get positions

In [ ]:
try:
    positions = trading_client.get_all_positions()
    positions = {x.symbol: x.qty for x in positions}
    positions = pd.Series(positions)
    df["current"] = positions
except:
    df["current"] = 0

## Trade

#### Calculate target positions

In [ ]:
long_per_stock = 1.3*equity / numstocks
short_per_stock = 0.3*equity / numstocks

df = df.sort_values(by="rnk")

try:
    short_cutoff = df[df.shortable & (df.bid>0)].rnk.iloc[numstocks-1]
    long_cutoff = df[df.tradable & (df.ask>0)].rnk.iloc[-numstocks]
    df["target"] = np.where(df.rnk<=short_cutoff, -short_per_stock/df.bid, 0)
    df["target"] = np.where(df.rnk>=long_cutoff, long_per_stock/df.ask, df.target)
    df["target"] = df.target.astype(int)
except:
    df["target"] = 0

#### Calculate trades

Using a simple but suboptimal protocol: trade to target positions without trying to minimize the number of round trips we might eventually make.

In [ ]:
df["trade"] = df.target - df.current

#### Make trades

In [ ]:
for tick in df.index: 
    if df.loc[tick, "trade"]<0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=-df.loc[tick, "trade"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"sell order for {tick} failed")
    elif df.loc[tick, "trade"]>0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=df.loc[tick, "trade"],
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"buy order for {tick} failed")



## Save data

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")

df["date"] = today

try:
    d = pd.read_csv("files/trade_data.csv", index_col="ticker")
    df = pd.concat((d, df))
    df.to_csv("files/trade_data.csv")
except:
    df.to_csv("files/trade_data.csv")

In [ ]:
account = trading_client.get_account()
account = dict(account)
account = pd.DataFrame(pd.Series(account)).T
account["date"] = today

try:
    d = pd.read_csv("files/account.csv")
    account = pd.concat((d, account))
    account.to_csv("files/account.csv")
except:
    account.to_csv("files/account.csv")


In [ ]:
positions = trading_client.get_all_positions()
positions = {x.symbol: x.qty for x in positions}
positions = pd.DataFrame(pd.Series(positions))
positions["date"] = today

try:
    d = pd.read_csv("files/positions.csv")
    positions = pd.concat((d, positions))
    positions.to_csv("files/positions.csv")
except:
    positions.to_csv("files/positions.csv")